# TensorBoard
TensorBoard is a tool for providing the measurements and visualizations needed during the machine learning workflow. It enables tracking experiment metrics like loss and accuracy, visualizing the model graph, projecting embeddings to a lower dimensional space, and much more.

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [6]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
   1/1875 [..............................] - ETA: 0s - loss: 2.2658 - accuracy: 0.1562WARNING:tensorflow:From C:\Users\Iryna.Dosiak\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2187 - accuracy: 0.9354 - val_loss: 0.1058 - val_accuracy: 0.9683
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0956 - accuracy: 0.9704 - val_loss: 0.0796 - val_accuracy: 0.9740
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0670 - accuracy: 0.9795 - val_loss: 0.0742 - val_accuracy: 0.9784
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0525 - accuracy: 0.9834 - val_loss: 0.0673 - val_accuracy: 0.9783
Epoch 5/5
1875/1875 [==============================]

In [7]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 11400), started 0:16:02 ago. (Use '!kill 11400' to kill it.)

##### The Scalars dashboard shows how the loss and metrics change with every epoch. You can use it to also track training speed, learning rate, and other scalar values.
##### The Graphs dashboard helps you visualize your model. In this case, the Keras graph of layers is shown which can help you ensure it is built correctly.
##### The Distributions and Histograms dashboards show the distribution of a Tensor over time. This can be useful to visualize weights and biases and verify that they are changing in an expected way.

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [10]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [11]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [12]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [13]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24627166986465454, Accuracy: 92.76499938964844, Test Loss: 0.11075568944215775, Test Accuracy: 96.59000396728516
Epoch 2, Loss: 0.10377299040555954, Accuracy: 96.91999816894531, Test Loss: 0.0847214013338089, Test Accuracy: 97.29999542236328
Epoch 3, Loss: 0.07246007025241852, Accuracy: 97.82333374023438, Test Loss: 0.06965361535549164, Test Accuracy: 97.8499984741211
Epoch 4, Loss: 0.054395757615566254, Accuracy: 98.3316650390625, Test Loss: 0.06543847173452377, Test Accuracy: 97.8499984741211
Epoch 5, Loss: 0.04331332817673683, Accuracy: 98.6500015258789, Test Loss: 0.059912294149398804, Test Accuracy: 98.18000030517578


In [14]:
%tensorboard --logdir logs/gradient_tape

Reusing TensorBoard on port 6006 (pid 14456), started 0:12:39 ago. (Use '!kill 14456' to kill it.)